# Installation

pip install -U keras-tuner

### Optional (Only if errors appear)

1. pip install tensorflow --upgrade --user
2. pip install keras --upgrade --user

In [1]:
import pandas as pd
import numpy as np

dataset=pd.read_csv('cardio_dataset.csv')
print(dataset.head())
dataset=dataset.values

   SEX  AGEIR   TC  HDL  SMOKE_  BPMED  DIAB_01  RISK
0    1     48  236   66       0      1        0   1.1
1    0     48  260   51       0      1        1   7.0
2    0     44  187   49       1      1        0   7.0
3    1     42  216   57       1      1        0   0.4
4    1     56  156   42       0      1        0   2.2


In [2]:
data=dataset[:,0:7]
target=dataset[:,7]

In [3]:
from sklearn.preprocessing import MinMaxScaler

target=np.reshape(target, (-1,1))

scaler_data = MinMaxScaler(feature_range=(0,1))
scaler_target = MinMaxScaler()

data_scaled=scaler_data.fit_transform(data)
target_scaled=scaler_target.fit_transform(target)

In [4]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(data_scaled, target_scaled,test_size=0.2)

In [5]:
from keras.models import Sequential
from keras.layers import Dense,Dropout
import numpy as np

from keras.optimizers import Adam,Adagrad,Adadelta

def build_model(parameters):
    
    model = Sequential()
    
    for i in range(parameters.Int('num_layers', 2, 20)):
#number of layers gana num_layers kynne.
#2-20 athra random num ekak thorla denwa.10 unth loop eka 10 parak run wnw.
        if(i==0):
        #palaweni iteration ekedi units wlt random value ekak denwa.
        #min 32 and max 512 step eka 32i ethkt 32 , 64 wge 32n 32t random value ekak gnnwa
        #parametr choice eke randomly relu sigmoid wge euwa gnnwa.
        #i=0 welwata wenma ekk declare krnne palaweni layer eke input dim gana denna oni nisa.
            model.add(Dense(units=parameters.Int('#neurons layer' + str(i),min_value=32,max_value=512,step=32),
                            activation=parameters.Choice('activation_function '+str(i),['relu','sigmoid','tanh']),input_dim=7))
        #drop out wltath random layer ekak denwa.    
            model.add(Dropout(parameters.Choice('drop_prob 1',[0.2,0.3,0.4,0.5])))
        
        else:
            #first hiden layer ekt passe euwa.
            model.add(Dense(units=parameters.Int('#neurons layer' + str(i),min_value=32,max_value=512,step=32),
                                   activation=parameters.Choice('activation_function '+str(i),['relu','sigmoid','tanh'])))
            
            model.add(Dropout(parameters.Choice('drop_prob 2',[0.2,0.3,0.4,0.5])))
    #last layer liner , regression problem nisa.        
    model.add(Dense(1, activation='linear'))
    
    #random optimizer 3k denwa ekak gnna.
    model.compile(optimizer=parameters.Choice('optmz', ['adam', 'adaDelta', 'adaGrad']),loss=parameters.Choice('loss f', ['mse', 'mae']))
    #model.compile(optimizer=Adam(parameters.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),loss='mse')
    
    return model

In [6]:
from kerastuner.tuners import RandomSearch

tuner = RandomSearch(build_model,objective='val_loss',max_trials=5,executions_per_trial=3,directory='project',
                     project_name='heart-risk')

#tuner means an object from tuner library
#build_model kiyanne model ek hdna function eka
#regression nisa val_loss e wtrk nwi mean squared error wge oni ekk use krnna puluwan
#max_trials means randomly neural netowrk kyk thornwda kyn eka.
#eka neural netowrk ek ki parak run krnwd kyn eka
#neural netork save wena thena folder name eka project
#hart rick next folder

[more info](https://keras-team.github.io/keras-tuner/)

In [7]:
tuner.search_space_summary()
#api train krnna yna model eke wenas wena dewak
#num_layers wge parameters tika.

Search space summary
Default search space size: 9
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
#neurons layer0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
activation_function 0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh'], 'ordered': False}
drop_prob 1 (Choice)
{'default': 0.2, 'conditions': [], 'values': [0.2, 0.3, 0.4, 0.5], 'ordered': True}
#neurons layer1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
activation_function 1 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'sigmoid', 'tanh'], 'ordered': False}
drop_prob 2 (Choice)
{'default': 0.2, 'conditions': [], 'values': [0.2, 0.3, 0.4, 0.5], 'ordered': True}
optmz (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'adaDelta', 'adaGrad'], 'ordered': False}
loss f (Choic

In [8]:
tuner.search(train_data,train_target,epochs=10,validation_data=(test_data, test_target))
#model . fit wage ekt meke tynne tuner.search use krnwa.

Trial 5 Complete [00h 00m 40s]
val_loss: 0.11139084647099178

Best val_loss So Far: 0.005032426056762536
Total elapsed time: 00h 03m 09s
INFO:tensorflow:Oracle triggered exit


In [9]:
tuner.results_summary()
#hodama model ekt enna oni dewal tynwa me summery eke.
#udinma tynne hodma eka. 
#ita pahalin ilgt hodma eka.

Results summary
Results in project\heart-risk
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
num_layers: 4
#neurons layer0: 96
activation_function 0: relu
drop_prob 1: 0.2
#neurons layer1: 288
activation_function 1: sigmoid
drop_prob 2: 0.4
optmz: adam
loss f: mse
#neurons layer2: 320
activation_function 2: tanh
#neurons layer3: 512
activation_function 3: relu
#neurons layer4: 160
activation_function 4: tanh
#neurons layer5: 192
activation_function 5: tanh
#neurons layer6: 192
activation_function 6: tanh
Score: 0.005032426056762536
Trial summary
Hyperparameters:
num_layers: 7
#neurons layer0: 160
activation_function 0: tanh
drop_prob 1: 0.5
#neurons layer1: 288
activation_function 1: relu
drop_prob 2: 0.2
optmz: adaGrad
loss f: mse
#neurons layer2: 32
activation_function 2: relu
#neurons layer3: 32
activation_function 3: relu
#neurons layer4: 32
activation_function 4: relu
#neurons layer5: 32
activation_function 5: relu
#neurons layer6

In [10]:
best_models = tuner.get_best_models()
print(best_models[0].summary())
#hodama model eke wistr pahadiliwa ganna puluwan

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 96)                768       
_________________________________________________________________
dropout (Dropout)            (None, 96)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 288)               27936     
_________________________________________________________________
dropout_1 (Dropout)          (None, 288)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 320)               92480     
_________________________________________________________________
dropout_2 (Dropout)          (None, 320)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               1